In [ ]:
# This code will create a DMD Matrix based on the X and Y (snapshot) matrix

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.sparse.linalg import eig, eigsh

In [ ]:
# Read Input
def read_variables_to_globals(filename):
    with open(filename, 'r') as f:
        lines = [line.strip() for line in f.readlines()]

    for i in range(0, len(lines), 2):
        var_name = lines[i]
        raw_value = lines[i+1]

        try:
            value = int(raw_value)
        except ValueError:
            try:
                value = float(raw_value)
            except ValueError:
                value = raw_value

        globals()[var_name] = value  # Define variable in global scope

In [ ]:
def DMD(ProjectName, NumSS, NumModes):
    # We need an A matrix, such that X = AY. where Y is a snapshot matrix defined as Y = X_{i+1}

    # A_tilde defined as operator that works in the column space of the snapshot matrix X.
    #   -> A_tilde = U^{T} * Y * V * Sigma^(-1)

    # Find Vt and Sigma of X (X has NumSS - 1 snaphots)
    corr_X = np.zeros((NumSS - 1, NumSS - 1))  # Initialize correlation matrix
    ModeSize = 0 # Intialize the size of each mode

    # Perform every dot product combo on the column snapshots in order to build the correlation matrix
    for i in range(NumSS-1):
        for j in range(i, NumSS-1):  # j starts at i
            snap_id_a = i + 1
            snap_id_b = j + 1

            snap_filename_a = ProjectName + "_SnapShot_"  + str(snap_id_a) + ".txt"
            snap_filename_b = ProjectName + "_SnapShot_"  + str(snap_id_b) + ".txt"

            snap_a = np.loadtxt(snap_filename_a)
            snap_b = np.loadtxt(snap_filename_b)

            if i == 0 and j == 0:
                ModeSize = len(snap_a)

            corr_X[i, j] = np.dot(snap_a, snap_b) 
            corr_X[j, i] = corr_X[i, j]       # fill symmetric entry

    eig_corr, tempModes = eigsh(corr_X, k=NumSS)
    print("Size of the eigenvalues")
    print(eig_corr.size)
    
    tempModes = tempModes[:, ::-1]  # Flip columns

    Vt_X = np.transpose(tempModes)
    S_X = np.sqrt(eig_corr)
    S_X = S_X[::-1]

    # Calculate U^{T} * Y in one loop (end result should be the size of (NumSS - 1) by (NumSS -1 ))

    UtY = np.zeros((NumSS-1, NumSS-1)) # Initialize UtY
    
    for k in range(NumSS-1): 
        mode_init = np.zeros(ModeSize)
        # Find each mode of X (columns of U)
        for i in range(NumSS-1):
            snapshot_id = i + 1
            snap_filename = ProjectName + "_SnapShot_"  + str(snapshot_id) + ".txt"
            snapshot = np.loadtxt(snap_filename)
            
            mode_init = mode_init + snapshot*Vt_X[k,i]*S_X[k]**(-1)

        # Use newly calculated mode to fill in the kth row of UtY 
        for i in range(NumSS-1):
            Y_ID = i + 2
            Ycol_filename = ProjectName + "_SnapShot_"  + str(Y_ID) + ".txt"
            Ycol = np.loadtxt(Ycol_filename)
            UtY[k,i] = np.dot(mode_init, Ycol)

    
    # Perform UtY * V * sigma^(-1)
    A_tilde = UtY*tempModes*np.diag(1/S_X)

    # Find Eigenvalues and eigenvectors of A_tilde to get the DMD modes and eigenvalues
    DMD_eig, DMD_modes = eig(A_tilde, k=NumSS-1)
    
    # Reinflate the modes into higher dimensional form
    
    return DMD_modes, DMD_eig

In [ ]:
# Plot complex eigenvalues on a unit circle

def plot_eigs_on_unit_circle(eigvals, b_k, n=1, cmap='viridis'):
    """
    Plot complex eigenvalues on the complex plane with a unit circle.
    
    Parameters
    ----------
    eigvals : np.ndarray
        Complex eigenvalues (1D array).
    b_k : np.ndarray
        Mode amplitudes (same length as eigvals).
    n : int
        Exponent for lambda in |lambda^n * b_k| (default: 1).
    cmap : str
        Colormap for points (default: 'viridis').
    """

    # Compute color values
    color_values = np.abs((eigvals**n) * b_k)

    fig, ax = plt.subplots(figsize=(6,6))

    # Plot unit circle
    theta = np.linspace(0, 2*np.pi, 500)
    ax.plot(np.cos(theta), np.sin(theta), 'k', linewidth=2)

    # Scatter plot of eigenvalues
    sc = ax.scatter(eigvals.real, eigvals.imag, 
                    c=color_values, cmap=cmap, s=100, edgecolors='k')

    # Labels and formatting
    ax.set_xlabel(r"Re($\lambda_k$)")
    ax.set_ylabel(r"Im($\lambda_k$)")
    ax.set_aspect('equal', adjustable='box')

    # Colorbar
    cbar = plt.colorbar(sc, ax=ax)
    cbar.set_label(r"$|\lambda_k^n b_k|$")

    plt.show()

In [ ]:
# Plot the eigenvalues against amountof energy they each contain

In [ ]:
# Plot temporal dynamics
def plot_dmd_temporal_modes(eigs, deltat, NumModes, num_time_steps=200):
    """
    Plots the temporal behavior of the first NumModes DMD eigenvalues.
    
    Parameters:
    - eigs: 1D numpy array of DMD eigenvalues (complex), length >= NumModes
    - deltat: time spacing between snapshots
    - NumModes: number of leading modes to plot
    - num_time_steps: number of points in the time array
    """
    t = np.linspace(0, (num_time_steps - 1) * deltat, num_time_steps)
    fig, axs = plt.subplots(NumModes, 1, figsize=(8, 2.5 * NumModes), sharex=True)

    if NumModes == 1:
        axs = [axs]  # wrap for consistency

    for i in range(NumModes):
        # Continuous-time eigenvalue
        omega_c = np.log(eigs[i]) / deltat
        sigma = omega_c.real      # growth/decay rate
        omega = omega_c.imag      # angular frequency (rad/s)

        # Real part of mode time evolution
        time_course = np.exp(sigma * t) * np.cos(omega * t)

        axs[i].plot(t, time_course, linewidth=2)
        axs[i].set_title(
            f"DMD Mode {i+1}: σ={sigma:.3f}, f={omega/(2*np.pi):.3f} Hz",
            fontsize=12
        )
        axs[i].set_ylabel("Amplitude")
        axs[i].grid(True)

    axs[-1].set_xlabel("Time")
    plt.tight_layout()
    plt.show()

In [ ]:
# Print Modes
def save_modes(ProjectName, DMD_modes):
    """
    Saves each column of the U matrix into separate text files.

    Parameters:
    - ProjectName (str): Base name for output files
    - U (numpy.ndarray): U matrix from economy SVD
    """

    # Number of columns in U
    num_cols = DMD_modes.shape[1]

    # Loop through columns
    for DMDmode_col_id in range(1, num_cols + 1):
        filename_out = f"{ProjectName}_dmd"
        "Mode_{DMDmode_col_id}.txt"
        
        # Extract column as a 2D array for saving
        col_data = DMD_modes[:, DMDmode_col_id - 1].reshape(-1, 1)
        
        # Save as text
        np.savetxt(filename_out, col_data, fmt="%.6f")
        print(f"Saved: {filename_out}")

In [ ]:
def main(ProjectName, NumSS, NumModes):
    

    return 